# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of native and semantic functions that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`. We encourage you to implement your own versions of the planner and use different models that fit your user needs.  

Read more about planner [here](https://aka.ms/sk/concepts/planner)

In [1]:
#!python -m pip install semantic-kernel==0.4.5.dev0

In [1]:
import os
endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
deployment = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]
embeddings = os.environ["AZURE_OPENAI_EMBEDDINGS_MODEL_NAME"]
azure_ai_search_api_key = os.environ["AZURE_AISEARCH_API_KEY"]
azure_ai_search_url = os.environ["AZURE_AISEARCH_ENDPOINT"]
#BING_API_KEY = os.environ["BING_API_KEY"]

In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    AzureChatCompletion,
)

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI backend used by the kernel
if useAzureOpenAI:
    
    kernel.add_chat_service(
        "chat_completion",
        AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key),
    )
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service(
        "gpt-3.5",
        OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id),
    )

## It all begins with an ask

In [3]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset. 

You can also include native functions as well. Here we'll add the TextSkill.

In [4]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../semantic-kernel/samples/skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

Define your ASK. What do you want the Kernel to do?

# Basic Planner

 Let's start by taking a look at a basic planner. The `BasicPlanner` produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.

In [12]:
from semantic_kernel.planning.basic_planner import BasicPlanner

planner = BasicPlanner()

In [13]:
basic_plan = await planner.create_plan_async(ask, kernel)

In [14]:
print(basic_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "ShakespeareSkill.shakespeare"},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


You can see that the Planner took my ask and converted it into an JSON-based plan detailing how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call in order to fulfill the user ask. The output of each step of the plan becomes the input to the next function.

Let's also define an inline skill and have it be available to the Planner. Be sure to give it a function name and skill name.

In [15]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="shakespeare",
    skill_name="ShakespeareSkill",
    max_tokens=2000,
    temperature=0.8,
)

Let's update our ask using this new skill

In [16]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan_async(ask, kernel)

In [17]:
print(new_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "ShakespeareSkill.shakespeare"},
        {"function": "WriterSkill.EmailTo", "args": {"recipient": "significant_other"}},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [18]:
results = await planner.execute_plan_async(new_plan, kernel)

Variable `$to` not found
Variable `$sender` not found


In [19]:
print(results)

CHER [DESTINATAIRE],

J'ESPÈRE QUE CET EMAIL VOUS TROUVE EN BONNE SANTÉ. JE VOULAIS PARTAGER AVEC VOUS QUELQUES IDÉES POUR PASSER DU TEMPS DE QUALITÉ ENSEMBLE, CAR JE PENSE QU'IL EST IMPORTANT DE NOURRIR NOTRE RELATION. VOICI QUELQUES SUGGESTIONS :

1. QUE DIRIEZ-VOUS D'UN DÎNER ROMANTIQUE DANS UN CHARMANT RESTAURANT, OÙ NOUS POURRIONS PROFITER DE LA COMPAGNIE DE L'AUTRE AU MILIEU DE LA DOUCE LUEUR DES BOUGIES ?

2. UNE AUTRE IDÉE SERAIT D'AVOIR UN CHARMANT PIQUE-NIQUE DANS UNE BELLE PRAIRIE OU AU BORD DE LA MER, OÙ NOUS POURRIONS LAISSER LES VAGUES ET NOTRE AMOUR NOUS GUIDER.

3. SI NOUS PRÉFÉRONS UNE SOIRÉE CONFORTABLE À LA MAISON, NOUS POURRIONS ORGANISER UN MARATHON DE FILMS, BLOTTIS SOUS DES COUVERTURES DANS UNE CABANE QUE NOUS CRÉERIONS ENSEMBLE.

4. POUR UNE JOURNÉE DE DÉTENTE ET DE BIEN-ÊTRE, NOUS POURRIONS NOUS OFFRIR UNE EXPÉRIENCE AU SPA, OÙ NOUS POURRIONS NOUS DÉTENDRE ET LAISSER TOMBER TOUT STRESS.

5. SI NOUS AVONS ENVIE D'AVENTURE, NOUS POURRIONS NOUS LANCER DANS UNE ESC

# The Plan Object Model

To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/plan.py)

# Sequential Planner

The sequential planner is an XML-based step-by-step planner. You can see the prompt used for it here (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Skills/SequentialPlanning/skprompt.txt)

In [20]:
from semantic_kernel.planning import SequentialPlanner

planner = SequentialPlanner(kernel)

In [21]:
sequential_plan = await planner.create_plan_async(goal=ask)

To see the steps that the Sequential Planner will take, we can iterate over them and print their descriptions

In [22]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Given a goal or topic description generate a list of ideas : {'variables': {'input': ''}}
Translate the input into a language of your choice : {'variables': {'input': ''}}
Turn a scenario into a short and entertaining poem. : {'variables': {'input': ''}}
Convert a string to uppercase. : {'variables': {'input': ''}}


Let's ask the sequential planner to execute the plan.

In [23]:
result = await sequential_plan.invoke_async()

In [24]:
print(result)

1. PIQUE-NIQUE DANS LE PARC
2. VISITER UN MUSÉE LOCAL OU UNE GALERIE D'ART
3. SUIVRE UN COURS DE CUISINE ENSEMBLE
4. FAIRE UNE RANDONNÉE OU UNE PROMENADE EN PLEINE NATURE
5. PASSER UNE SOIRÉE CINÉMA À LA MAISON AVEC DU POPCORN FAIT MAISON
6. EXPLORER UNE VILLE OU UN VILLAGE À PROXIMITÉ ET ESSAYER DE NOUVEAUX RESTAURANTS
7. ASSISTER À UN CONCERT DE MUSIQUE LIVE OU À UN SPECTACLE DE COMÉDIE
8. FAIRE UNE BALADE PANORAMIQUE ET PIQUE-NIQUER À UN POINT DE VUE
9. ORGANISER UNE SOIRÉE JEUX AVEC DES JEUX DE SOCIÉTÉ OU DES JEUX VIDÉO
10. VISITER UN MARCHÉ FERMIER LOCAL ET CUISINER UN REPAS ENSEMBLE EN UTILISANT DES INGRÉDIENTS FRAIS


# Action Planner

The action planner takes in a list of functions and the goal, and outputs a **single** function to use that is appropriate to meet that goal.

In [25]:
from semantic_kernel.planning import ActionPlanner

planner = ActionPlanner(kernel)

Let's add more skills to the kernel

In [26]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill

kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

{'lowercase': SKFunction(),
 'trim': SKFunction(),
 'trim_end': SKFunction(),
 'trim_start': SKFunction(),
 'uppercase': SKFunction()}

In [27]:
ask = "What is the sum of 110 and 990?"

In [28]:
plan = await planner.create_plan_async(goal=ask)

In [29]:
result = await plan.invoke_async()

In [30]:
print(result)

1100


# Stepwise Planner

Stepwise Planner is based off the paper from MRKL (Modular Reasoning, Knowledge and Language) and is similar to other papers like ReACT (Reasoning and Acting in Language Models). At the core, the stepwise planner allows for the AI to form "thoughts" and "observations" and execute actions based off those to achieve a user's goal. This continues until all required functions are complete and a final output is generated.

See a video walkthrough of Stepwise Planner [here.](https://youtu.be/DG_Ge1v0c4Q?si=T1CHaAm1vV0mWRHu)

In [31]:
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import (
    StepwisePlannerConfig,
)

Let's create a Bing Search native skill that we can pass in to the Kernel.

Make sure you have a Bing Search API key in your `.env` file

(https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)

In [32]:
class WebSearchEngineSkill:
    """
    A search engine skill.
    """

    from semantic_kernel.orchestration.sk_context import SKContext
    from semantic_kernel.skill_definition import (
        sk_function,
        sk_function_context_parameter,
    )

    def __init__(self, connector) -> None:
        self._connector = connector

    @sk_function(description="Performs a web search for a given query", name="searchAsync")
    @sk_function_context_parameter(
        name="query",
        description="The search query",
    )
    async def search_async(self, query: str, context: SKContext) -> str:
        query = query or context.variables.get("query")[1]
        result = await self._connector.search_async(query, num_results=5, offset=0)
        return str(result)

In [33]:
from semantic_kernel.connectors.search_engine import BingConnector

BING_API_KEY = "debc8bdb85944e54b3199eec4213bf55"
connector = BingConnector(BING_API_KEY)
kernel.import_skill(WebSearchEngineSkill(connector), skill_name="WebSearch")

{'searchAsync': SKFunction()}

Let's also add a couple more skills

In [34]:
from semantic_kernel.core_skills.math_skill import MathSkill
from semantic_kernel.core_skills.time_skill import TimeSkill

kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(MathSkill(), "math")

{'Add': SKFunction(), 'Subtract': SKFunction()}

In [35]:
planner = StepwisePlanner(kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000))

Now let's do a more complicated ask that will require planner to make a call to Bing to get the latest information.

In [36]:
ask = """What is the Argentinian inflation rate right now?"""

plan = planner.create_plan(goal=ask)

In [37]:
for step in plan._steps:
    print(step.description, ":", step._state.__dict__)

Execute a plan : {'variables': {'input': ''}}


In [38]:
result = await plan.invoke_async()

Variable `$agent_scratch_pad` not found


In [39]:
print(result)

I'm sorry, but I cannot provide the current Argentinian inflation rate as it requires an internet search. Please use the "WebSearch.searchAsync" function with the query "current Argentinian inflation rate" to get the latest information.


Let's see the steps that the AI took to get to the answer.

In [40]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:", step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print("  Output:\n", str.replace(result[step._outputs[0]], "\n", "\n  "))

Step: 0
Description: Execute a plan
Function: StepwisePlanner.ExecutePlan
  Output:
 This was my previous work (but they haven't seen any of it! They only see what I return as final answer):
  [THOUGHT]
  None


# Conclusion

- Planner and Sequential-Planner works in the documentation example
- Sequential-Planner fail many times, in one experimt due to OpenAI-gpt token quota, and other strange answer.